In [1]:
import LLM
from SDMX_DataFlow import Dataflow
from data_prep_for_indenxing import flatten_info

In [2]:
# Define the dataflow URL <= this will come from the dropdown selection of the chatbot
_fin_perstud = {"agency": "OECD.EDU.IMEP",
"id": "DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD",
"version": "1.0"}

fin_perstud = {"agency": "OECD.EDU.IMEP",
"id": "DSD_EAG_UOE_FIN@DF_UOE_FIN_INDIC_SOURCE_NATURE",
"version": "3.0"}

dataflow_details_url = f'https://sdmx.oecd.org/public/rest/dataflow/{fin_perstud["agency"]}/{fin_perstud["id"]}/{fin_perstud["version"]}?references=all'

# Create an instance of the Dataflow class and populate the variables
df_info = Dataflow(dataflow_details_url)

print("Populating variables...")
df_info.populate_variables()
print("Variables populated.")

Populating variables...
Variables populated.


In [3]:
# Flatten the dataflow information for embedding
flat_info_for_embedding = flatten_info(df_info)

INTENSITY not found in the code names.
INST_TYPE_EDU not found in the code names.
BASE_PER not found in the code names.
TIME_PERIOD not found in the code names.
OBS_VALUE not found in the code names.
OBS_STATUS not found in the code names.
UNIT_MULT not found in the code names.
DECIMALS not found in the code names.
Q_SHEET_ROW_ID not found in the code names.


In [4]:
from chromaDB import ChromaDBWrapper
# Create the document store
print("Creating the document store...")
chroma_wrapper = ChromaDBWrapper(flat_info_for_embedding) # should be called with the api key (since i already need it for the LLM)

Creating the document store...
Collection dataflow-meta-information-embeddings created successfully
703 embeddings created from documents were and added to the vector store.


In [5]:
chroma_wrapper.query('what does the code: "ISCED11_34_44" mean?', n_results=3)

{'ids': [['2cee6d2b-9573-4682-a55c-b31111f02e63',
   '3f75872e-6547-4f2d-bf2d-d23dab9b6516',
   '8b53a698-47a4-4b26-958f-d40f40d6af05']],
 'distances': [[0.47795817255973816, 0.4863402545452118, 0.490774929523468]],
 'metadatas': [[{'answer': "The English name of the code 'ISCED11_44' within the code list ID 'EDUCATION_LEV' is 'Post-secondary non-tertiary general education'."},
   {'answer': "The English name of the code 'ISCED11_34_44' within the code list ID 'EDUCATION_LEV' is 'Upper secondary and post-secondary non-tertiary general programmes'."},
   {'answer': "The English name of the code 'ISCED11_34' within the code list ID 'EDUCATION_LEV' is 'Upper secondary general education'."}]],
 'embeddings': None,
 'documents': [["What is the English name of the code 'ISCED11_44' within the code list ID 'EDUCATION_LEV'?",
   "What is the English name of the code 'ISCED11_34_44' within the code list ID 'EDUCATION_LEV'?",
   "What is the English name of the code 'ISCED11_34' within the code 

In [6]:
def answer_question(user_question, chroma_wrapper):
    # Get the most similar question and answer from the vector store
    result_sets = chroma_wrapper.query(user_question, n_results=3)

    # Generate the answer to the user question
    persona = """You are a helpful data analyst working for OECD."""
    prompt = f"""
    Please provide the answer to the following user question: 
    {user_question}
    Please use the information from the source that was selected as the best source to answer the question!
    This is the most similar question to the user question:
    '''{result_sets['documents'][0]}'''
    
    The answer to the question is:
    '''{result_sets['metadatas'][0][0]['answer']}'''
    """
    print(result_sets['documents'][0])
    print("")
    print(result_sets['metadatas'][0][0]['answer'])

    ans, cost = LLM.model(persona, prompt)
    return ans

In [14]:
answer_question("what is the  name of the dataframe", chroma_wrapper)

["The DataFrame's name", "The DataFrame's description", 'What are the columns or data structure components (dimensions) in this DataFlow?']

Full dataset - Indicators, source, destination and nature of expenditure on education


'The name of the dataframe is: **Full dataset - Indicators, source, destination and nature of expenditure on education**.'

In [ ]:
answer_question('what does the code: "ISCED11_34_44" mean?', chroma_wrapper)

In [ ]:
answer_question('what dimensions are present in this dataflow?', chroma_wrapper)

In [ ]:
answer_question('what is the name of this dataflow?', chroma_wrapper)

In [1]:
import grounded_llm

In [ ]:
bot_instance = grounded_llm.Bot("OECD.EDU.IMEP:DSD_EAG_UOE_FIN@DF_UOE_FIN_INDIC_SOURCE_NATURE(3.0)") 

In [ ]:
bot_instance.answer_question('what does the code: "ISCED11_34_44" mean?')

In [ ]:
bot_instance.answer_question('what is the name of the dataflow?')

In [ ]:
bot_instance.answer_question('what dimensions are present in this dataflow?')


In [17]:
import LLM

In [19]:
def rephrase_user_question(user_question):
    """
    Before performing the semantic search it is wise to rephrase the user question to improve the search results.
    The idea is to provide consistency in the questions asked instead of having to store all possible ways of asking the same question.
    """

    persona = """You are a helpful data analyst working for OECD."""
    prompt = f"""
    Your task is to rephrase the user question to improve the search results.
    
    Please follow the steps above when rephrasing user questions into a simple and uniform format:
        1. Identify the Core Inquiry: Determine the main point of the question (e.g., dataset name, column presence, code meaning).
        2. Start with a Question Word: Use "What" or "Is" to begin the question.
        3. Focus on the Subject: Clearly state the subject of the question (e.g., dataset, column, code).

    Make sure to:
        - Use Simple Language: Avoid jargon and complex terms.
        - Be Specific: Provide details that help narrow down the search.
    
    In order to further decrease the complexity of the vocabulary used in the user questions, the following rules should be applied:
        - any synonyms to the data source in question (e.g. data, DataFlow, Data Source, Table, Data Set, Data Flow etc.) are replaced with "Table"
        - any synonyms of a Column are replaced with "Column" (e.g. "Variable", "Indicator", "Field", "Attribute", "Measure", "Dimension", "Concept", etc.)
        - any synonyms of Category are replaced with "Category" (e.g. "Code", "Label", "Code Value", etc.). Category is used to refer to the discrete values of a column. 

    Please rephrase the following user question:
    {user_question}
    """

    rephrased_question, cost = LLM.model(persona, prompt)

    return rephrased_question

In [20]:
natural_variations = [
    "What is the name of the data table?",
    "What is the name of the dataflow?",
    "How is the dataset called?",
    "What is the table's name?",
    "Can you tell me the name of the data table?",
    "What's the dataset's name?",
    "What is the official name of the data table?",
    "How is the data table referred to?",
    "What do we call this data table?",
    "What's the title of the data table?"
]

for question in natural_variations:
    print(rephrase_user_question(question))

What is the name of the Table?
What is the name of the Table?
What is the name of the Table?
What is the name of the Table?
What is the name of the Table?
What is the Table's name?
What is the official name of the Table?
What is the name of the Table?
What is the name of this Table?
What is the title of the Table?


In [21]:
def flatten_info(info):
    """Flatten the dataflow information for embedding."""
    flat_info_for_embedding = list(tuple())
    flat_info_for_embedding.extend(flatten_name_and_description(info))
    flat_info_for_embedding.extend(flatten_dimensions(info))
    flat_info_for_embedding.extend(flatten_codes(info))
    flat_info_for_embedding.extend(get_dataflow_struct_questions(info))

    return flat_info_for_embedding


def flatten_name_and_description(info):

    return [
          ("The Table's name", info.df_name)
        , (info.df_name, "The Table's name")
        , ("The Table's description", info.df_description)
        , (info.df_description, "The Table's description")
    ]


def flatten_dimensions(info):
    ans = []
    for code, name in info.df_dimension_names.items():
        meta_statement = f"The name that corresponds to the column code: '{code}' is {name}."
        ans.append((code, meta_statement))
        ans.append((name, meta_statement))
        ans.append((f"What name corresponds to the column code: '{code}'?", meta_statement))
        ans.append((f"What is the column code for '{name}'?", meta_statement))
    return ans


def flatten_codes(info):
    ans = []
    for code_list_id in info.df_code_names:
        for code, name in info.df_code_names[code_list_id].items():
            meta_statement = f"The English name of the code '{code}' within the code list ID '{code_list_id}' is '{name}'."
            ans.append((code, meta_statement))
            ans.append((name, meta_statement))
            ans.append((f"What is the English name of the code '{code}' within the code list ID '{code_list_id}'?", meta_statement))
            ans.append((f"What is the code for '{name}' within the code list ID '{code_list_id}'?", meta_statement))
    return ans


def get_dataflow_struct_questions(info):
    """
    If we want generic questions about the schema to be searchable, we need to put them explicitly in the vectorstore. 
    The two example use case here are when, the user asks for:
        1. All the columns in the data table.
        2. All the categories (codes) in a specific column.
    """

    ans = []
    ans.append((   "What are the columns in this Tables?"
                , f"These are all the dimension codes and associated English names in this DataFlow: {info.df_dimension_names}"))
   
    for dim_code, dim_name in info.df_dimension_names.items():
        try:
            ans.append((  f"What are all the categories in the column: '{dim_code}'?"
                        , f"""All codes and their English names corresponding to the dimension code: '{dim_code}' and dimension name '{dim_name}' are the follworing: 
                          '''{info.df_code_names[dim_code]}'''.
                          """))
            ans.append((  f"What are all the categories in the column: '{dim_name}'?"
                        , f"""All codes and their English names corresponding to the dimension code: '{dim_code}' and dimension name '{dim_name}' are the follworing: 
                          '''{info.df_code_names[dim_code]}'''.
                          """))
        except:
            print(f"{dim_code} not found in the code names.")

    return ans

# Testing the whole setup

In [22]:
# Define the dataflow URL <= this will come from the dropdown selection of the chatbot
fin_perstud = {"agency": "OECD.EDU.IMEP",
"id": "DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD",
"version": "1.0"}

dataflow_details_url = f'https://sdmx.oecd.org/public/rest/dataflow/{fin_perstud["agency"]}/{fin_perstud["id"]}/{fin_perstud["version"]}?references=all'

In [23]:
def load_dataflow_info_to_vector_store(dataflow_details_url):
    # Create an instance of the Dataflow class and populate the variables
    df_info = Dataflow(dataflow_details_url)

    print("Populating variables...")
    df_info.populate_variables()
    print("Variables populated.")

    # Flatten the dataflow information for embedding
    flat_info_for_embedding = flatten_info(df_info)

    from chromaDB import ChromaDBWrapper
    # Create the document store
    print("Creating the document store...")
    chroma_wrapper = ChromaDBWrapper(flat_info_for_embedding) # should be called with the api key (since i already need it for the LLM)

    return chroma_wrapper

In [24]:
my_chroma_wrapper = load_dataflow_info_to_vector_store(dataflow_details_url)

Populating variables...
Variables populated.
INTENSITY not found in the code names.
INST_TYPE_EDU not found in the code names.
BASE_PER not found in the code names.
TIME_PERIOD not found in the code names.
OBS_VALUE not found in the code names.
OBS_STATUS not found in the code names.
UNIT_MULT not found in the code names.
DECIMALS not found in the code names.
Creating the document store...
Collection dataflow-meta-information-embeddings created successfully
449 embeddings created from documents were and added to the vector store.


In [ ]:
user_question = "what is the  name of the dataframe"
standardised_user_question = rephrase_user_question(user_question)
print(f'{standardised_user_question=}')
answer_question(standardised_user_question, my_chroma_wrapper)

In [7]:
import json
# read grouped_edu_dataflows.json
with open('grouped_edu_dataflows.json', 'r') as f:
    dataflows = json.load(f)

In [ ]:
dataflows.keys()

In [ ]:
dataflows['UOE Financial']

In [ ]:
def setup_vector_store(dataflow_name):
    dataflow_details_url = get_dataflow_url_from_name(dataflow_name)

    # Create an instance of the Dataflow class and populate the variables
    df_info = Dataflow(dataflow_details_url)

    print("Populating variables...")
    df_info.populate_variables()
    print("Variables populated.")

    # Flatten the dataflow information for embedding
    flat_info_for_embedding = flatten_info(df_info)

    # Create the document store
    print("Creating the document store...")
    chroma_wrapper = ChromaDBWrapper(flat_info_for_embedding) # should be called with the api key (since i already need it for the LLM)

    return chroma_wrapper

In [ ]:
def get_dataflow_url_from_name(dataflow_name):
    # Split the dataflow_name into agency and the rest
    agency, rest = dataflow_name.split(':')
    
    # Split the rest into id and version
    id_part, version = rest.split('(')
    
    # Remove the closing parenthesis from the version
    version = version.rstrip(')')
    
    # Return the dataflow url
    return f'https://sdmx.oecd.org/public/rest/dataflow/{agency}/{id_part}/{version}?references=all'


dataflow_name = "OECD.EDU.IMEP:DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD(3.0)"
dataflow_details_url = get_dataflow_url_from_name(dataflow_name)
dataflow_details_url

In [13]:
from SDMX_DataFlow import Dataflow

In [ ]:
# Create an instance of the Dataflow class and populate the variables
df_info = Dataflow(dataflow_details_url)

print("Populating variables...")
df_info.populate_variables()
print("Variables populated.")

In [19]:
from data_prep_for_indenxing import flatten_info

# Flatten the dataflow information for embedding
flat_info_for_embedding = flatten_info(df_info)

In [ ]:
# Create the document store
print("Creating the document store...")
chroma_wrapper = ChromaDBWrapper(flat_info_for_embedding)

In [15]:
def answer_question(user_question, chroma_wrapper):
    # Get the most similar question and answer from the vector store
    result_sets = chroma_wrapper.query(user_question, n_results=3)

    # Generate the answer to the user question
    persona = """You are a helpful data analyst working for OECD."""
    prompt = f"""
    Please provide the answer to the following user question: 
    {user_question}
    Please use the information from the source that was selected as the best source to answer the question!
    This is the most similar question to the user question:
    '''{result_sets['documents'][0]}'''
    
    The answer to the question is:
    '''{result_sets['metadatas'][0][0]['answer']}'''
    """
    print(result_sets['documents'][0])
    print("")
    print(result_sets['metadatas'][0][0]['answer'])

    ans, cost = LLM.model(persona, prompt)
    return ans

In [ ]:
answer_question('what does the code: "ISCED11_34_44" mean?', chroma_wrapper)

In [ ]:
chroma_wrapper.query('what does the code: "ISCED11_34_44" mean?', n_results=3)

In [ ]:
chroma_wrapper

In [ ]:
import LLM
from SDMX_DataFlow import Dataflow
from data_prep_for_indenxing import flatten_info
from chromaDB import ChromaDBWrapper




# Create an instance of the Dataflow class and populate the variables
df_info = Dataflow(dataflow_details_url)

print("Populating variables...")
df_info.populate_variables()
print("Variables populated.")

# Flatten the dataflow information for embedding
flat_info_for_embedding = flatten_info(df_info)

# Create the document store
print("Creating the document store...")
chroma_wrapper = ChromaDBWrapper(flat_info_for_embedding) # should be called with the api key (since i already need it for the LLM)

def answer_question(user_question):
    # Get the most similar question and answer from the vector store
    result_sets = chroma_wrapper.query(user_question, n_results=1)

    # Generate the answer to the user question
    persona = """You are a helpful data analyst working for OECD."""
    prompt = f"""
    Please provide the answer to the following user question: 
    {user_question}
    Please use the information from the source that was selected as the best source to answer the question!
    This is the most similar question to the user question:
    '''{result_sets['documents'][0]}'''
    
    The answer to the question is:
    '''{result_sets['metadatas'][0][0]['answer']}'''
    """

    ans, cost = LLM.model(persona, prompt)
    return ans
